# Model Comparison

## TODO

1. make a for loop to go over each model, to avoid typing out everything multiple times
2. Make each model have the scoring and confusion matrix, and the validation plot
3. Learn about the validation plot


---

### This is gridsearch for each model

https://stackabuse.com/grid-search-optimization-algorithm-in-python/

In [13]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
import time
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
from pprint import pprint
from xgboost import XGBClassifier

In [23]:
from util import databases
from util import dataloader
from util import grid_models
from util.reddit_functions import Labeler
from util.reddit_functions import plot_confusion_matrix
from util.grid_models import custom_stop_words

In [15]:
# subreddit_list = ['css', 'html', 'javascript', 'php', 'perl', 'java', 'datascience', 'machinelearning', 'etl', 'python', 'dataengineering']

In [16]:
subreddit_list = ['datascience','machinelearning','dataengineering','python','aws']

In [17]:
df = dataloader.data_selector(subreddit_list, 'sqlite')

Connection to SQLite DB successful


In [18]:
df.sample(10)

,title,subreddit,date
26204,[D] What is the current approach for image cla...,machinelearning,2020-04-21
19073,"“Oh, you’re a data… something?”: The Misunders...",dataengineering,2020-04-14
48467,Can't run mysql with an EBS volume in EKS kube...,aws,2020-04-27
12289,[P] Follow-up work on the baikal library,machinelearning,2020-04-10
43031,"AWS is deprecating its pricing calculator, so ...",aws,2020-04-25
2234,[Discussion] Convolutional Variational Autoenc...,machinelearning,2020-03-29
7329,[D] Socially Beneficial Uses of DeepFakes,machinelearning,2020-04-02
27155,covid report for nation state and county from ...,python,2020-04-21
29366,Question about creating an EC2 instance that c...,aws,2020-04-22
6382,How to learn Data Science???,datascience,2020-04-02


In [20]:
X = df['title']
y = df['subreddit']

In [21]:
labeler = Labeler()
labeler.fit(y)
y = labeler.transform(y)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)

In [ ]:
tfidf = TfidfVectorizer(stop_words=custom_stop_words)

tfidf_params = 
    "tfidf__ngram_range": [(1, 2)],
#     "tfidf__max_features": [1000, 3000, 5000],
    "tfidf__max_df": [.75, .8, .85, .9],
    "tfidf__use_idf": [True],
    "tfidf__norm": ["l1", "l2"],

In [1]:
def build_and_train_model(preprocessor, classifier, pipeline, pipe_params, cv=3, verbose=1):
    '''
    Takes a dictionary with params and outputs a fitted model
    '''
    pipe = Pipeline(
    [('preprocessor', pre),
     ('classifier', clf)])
    
    model = GridSearchCV(pipe, param_grid=pipe_params, cv=cv, verbose=verbose, n_jobs=-1)
    model.fit(X_train, y_train)
    return model

In [27]:
def score_model(model):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)
    
    print(f'Train Score: {model.score(X_train, y_train)}')
    print(f'Test Score: {model.score(X_test, y_test)}')
    print(f'AUC Score: {roc_auc_score(y_test, y_proba)}')
    print(classification_report(y_test, y_pred, digits=3))

In [26]:
mlp = {
    'clf': MLPClassifier(),
    'clf_params': {
        "mlp__hidden_layer_sizes": [50, 100, 200]
    }
}

In [ ]:
xgb = {
    'clf': XGBClassifier(),
    'clf_params': {
        "xgb__hidden_layer_sizes": [10, 25, 50],
        "xgb__n_estimators": [50, 100, 200],
        "xgb__max_depth": [5, 10, 20]
    }
}

In [ ]:
onevrest = {
    'clf': OneVsRestClassifier(RandomForestClassifier()),
    'clf_params': {
        "tfidf__ngram_range": [(1, 2)],
        "tfidf__max_features": [4500, 5000, 5500],
        "tfidf__max_df": [.75, .8, .85],
        "tfidf__use_idf": [True],
        "tfidf__norm": ["l2"],
        "onevrest__estimator__n_estimators": [200, 300]
    }
}

In [ ]:
from sklearn.learning_curve import validation_curve
degree = np.arange(0, 21)
train_score, val_score = validation_curve(PolynomialRegression(), X, y,
'polynomialfeatures__degree', degree, cv=7)
plt.plot(degree, np.median(train_score, 1), color='blue', label='training score') plt.plot(degree, np.median(val_score, 1), color='red', label='validation score') plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('degree') plt.ylabel('score');

## Scoring

In [30]:
plot_confusion_matrix

In [ ]:
mcm = multilabel_confusion_matrix(y_test, y_pred)
mtn = mcm[:, 0, 0]
mtp = mcm[:, 1, 1]
mfn = mcm[:, 1, 0]
mfp = mcm[:, 0, 1]
print(mcm)

### ROC AUC Score

<h1>TODO:</h1>

1. Make it have the roc for each sub, have to get into the original df where subname equals indexes?
2. Plot confusion matrix
3. Make a notebook to test the confusion matrixes one by one with each individual model.

In [ ]:
prob_df = pd.DataFrame(y_prob, columns=subreddit_list)
prob_df.round(6)

In [ ]:

auc